In [4]:
import os
import pickle
import numpy as np
from queries import Simbad
from astropy.io import fits
from astropy.coordinates import SkyCoord
from IPython.display import clear_output

import sys
if sys.platform=='darwin':
    top_path='/Users/codydirks/PGCC/'
else:
    top_path='/DataDisk/datafiles/PGCC_HST/'

x1d_dir=top_path+'x1d_files/'

def find_in_tgas(key,field):
    filenames=[top_path+'gaia_data/TgasSource_000-000-0'+'{:02}'.format(i)+'.fits' for i in range(16)]
    for filename in filenames:
        gaia_data=fits.open(filename)[1].data
        #gaia_entry=gaia_data[gaia_data[field]==key]
        gaia_entry=[i for i,item in enumerate(gaia_data[field]==key) if item==True]
        if len(gaia_entry)>0:
            return filenames.index(filename),gaia_entry[0]
    return None
        
#Returns angular separation between two Ra/Dec coordinates in degrees
def get_ang_sep(ra1,dec1,ra2,dec2):
    ra1_rad=ra1*np.pi/180.
    dec1_rad=dec1*np.pi/180.
    ra2_rad=ra2*np.pi/180.
    dec2_rad=dec2*np.pi/180.

    del_ra=ra2_rad-ra1_rad
    del_ras=np.sin(del_ra)
    del_rac=np.cos(del_ra)
    dec1s=np.sin(dec1_rad)
    dec2s=np.sin(dec2_rad)
    dec1c=np.cos(dec1_rad)
    dec2c=np.cos(dec2_rad)
    result=(180./np.pi)*np.arctan(np.sqrt((dec2c**2)*(del_ras**2)+(dec1c*dec2s-dec1s*dec2c*del_rac)**2)/(dec1s*dec2s+dec1c*dec2c*del_rac))
    if result < 0:
        return result+180.
    else:
        return result
    
    
    
def store_results(results):
    filename='sightline_pgcc_gaia_results.txt'
    with open(filename,'w') as myfile:
        for entry in results:
            myfile.write('|'.join([entry[0],
                                   str(entry[1][0].value)+' '+str(entry[1][1].value),
                                   str(entry[2]),
                                   str(entry[3]),
                                   '\n']))
        

In [5]:
x1d_path=top_path+'x1d_files/'
sightlines=[x for x in os.listdir(x1d_path) 
            if os.path.isdir(x1d_path+x+'/E140H')==True and x.startswith('.')==False]
print sightlines
pgcc_data=fits.open(top_path+'HFI_PCCS_GCC_R2.02.fits')[1].data

['CPD-592603', 'HD040893', 'HD100546', 'HD101131', 'HD101436', 'HD10250', 'HD104705', 'HD106111', 'HD106343', 'HD106943', 'HD108002', 'HD108610', 'HD108639', 'HD108927', 'HD110434', 'HD111934', 'HD112999', 'HD114886', 'HD115071', 'HD115455', 'HD116781', 'HD117111', 'HD122879', 'HD124314', 'HD124979', 'HD137595', 'HD1383', 'HD13841', 'HD138679', 'HD147683', 'HD14818', 'HD148594', 'HD148937', 'HD15137', 'HD152249', 'HD152424', 'HD152590', 'HD157857', 'HD165918', 'HD173297', 'HD175640', 'HD177989', 'HD185418', 'HD186688', 'HD191877', 'HD192577', 'HD202347', 'HD203374', 'HD203532', 'HD206697', 'HD207198', 'HD208440', 'HD208816', 'HD208947', 'HD209339', 'HD210809', 'HD210839', 'HD21551', 'HD218915', 'HD219134', 'HD23180', 'HD232522', 'HD23478', 'HD23873', 'HD24398', 'HD24534', 'HD25443', 'HD254755', 'HD32630', 'HD36285', 'HD36959', 'HD37021', 'HD37061', 'HD37367', 'HD37903', 'HD40893', 'HD42111', 'HD43582', 'HD44173', 'HD46300', 'HD48915', 'HD50138', 'HD51285', 'HD52266', 'HD52329', 'HD6254

In [6]:
sightline_pgcc_gaia_data=[]

for sightline in sightlines:
    #clear_output()
    #print sightlines.index(sightline)+1, '/', len(sightlines)
    print sightline
    coords=Simbad.Position(sightline)
    nearest=9999
    nearest_pgcc=[]
    for i in range(len(pgcc_data)):
        pgcc_ra,pgcc_dec=pgcc_data[i][3:5]
        if abs(coords[0].value-pgcc_ra)<10 and abs(coords[1].value-pgcc_dec)<10:
            angsep=get_ang_sep(coords[0].value,coords[1].value,pgcc_ra,pgcc_dec)
            if angsep<nearest:
                nearest=angsep
                nearest_pgcc=i
    
    print nearest_pgcc
    
    idlist=Simbad.IDList(sightline)
    hip=[x for x in idlist if x.startswith('HIP')]
    tyc=[x for x in idlist if x.startswith('TYC')]
    if len(hip)>0 or len(tyc)>0:
        if len(hip)>0:
            hip_id=int(hip[0].split()[1])
            gaia_entry=find_in_tgas(hip_id,'hip')
        elif len(tyc)>0:
            tyc_id=tyc[0].split()[1]
            gaia_entry=find_in_tgas(tyc_id,'tycho2_id')
    print gaia_entry
    sightline_pgcc_gaia_data.append([sightline,coords,nearest_pgcc,gaia_entry])


CPD-592603
6916
(10, 105591)
HD040893
11863
(7, 48368)
HD100546
3184
(10, 15671)
HD101131
12852
(10, 84012)
HD101436
12852
(10, 84007)
HD10250
10571
None
HD104705
9814
(13, 89345)
HD106111
12770
(12, 108770)
HD106343
1551
(13, 84559)
HD106943
12249
(13, 89610)
HD108002
7950
(12, 114167)
HD108610
10204
(13, 86612)
HD108639
12207
(13, 90347)
HD108927
3080
(12, 56418)
HD110434
3685
(12, 113087)
HD111934
12653
(13, 88212)
HD112999
5813
(13, 87420)
HD114886
11162
None
HD115071
12137
(12, 115419)
HD115455
12137
(12, 123191)
HD116781
7444
(12, 122796)
HD117111
12196
(12, 116533)
HD122879
6425
(12, 122260)
HD124314
9956
(12, 107600)
HD124979
11123
(13, 12480)
HD137595
7862
(14, 8763)
HD1383
5473
(0, 119216)
HD13841
1720
(0, 133397)
HD138679
11558
(12, 83741)
HD147683
3412
None
HD14818
3783
(0, 132557)
HD148594
2816
None
HD148937
9849
(13, 39917)
HD15137
122
(0, 128920)
HD152249
3734
(13, 52030)
HD152424
3734
(13, 52007)
HD152590
234
(13, 53032)
HD157857
3337
None
HD165918
2451
None
HD173297
86

In [7]:
store_results(sightline_pgcc_gaia_data)